In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [6]:
spark = SparkSession.builder \
    .appName("Movie Analysis") \
    .getOrCreate()

24/04/17 01:21:15 WARN Utils: Your hostname, pranavs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.86.1.243 instead (on interface en0)
24/04/17 01:21:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/17 01:21:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/17 01:21:29 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
movies_df = spark.read.option("header", True).csv("/Users/pranavkumar/Downloads/movies.csv")


In [17]:
genres_df = movies_df.withColumn("genre", explode(split(col("genres"), "\\|")))
genres_count = genres_df.groupBy("genre").count()

In [11]:
genres_df.show()

+-------+--------------------+--------------------+---------+
|movieId|               title|              genres|    genre|
+-------+--------------------+--------------------+---------+
|      1|    Toy Story (1995)|Adventure|Animati...|Adventure|
|      1|    Toy Story (1995)|Adventure|Animati...|Animation|
|      1|    Toy Story (1995)|Adventure|Animati...| Children|
|      1|    Toy Story (1995)|Adventure|Animati...|   Comedy|
|      1|    Toy Story (1995)|Adventure|Animati...|  Fantasy|
|      2|      Jumanji (1995)|Adventure|Childre...|Adventure|
|      2|      Jumanji (1995)|Adventure|Childre...| Children|
|      2|      Jumanji (1995)|Adventure|Childre...|  Fantasy|
|      3|Grumpier Old Men ...|      Comedy|Romance|   Comedy|
|      3|Grumpier Old Men ...|      Comedy|Romance|  Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|   Comedy|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|    Drama|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|  Romance|
|      5

In [12]:
movies_df = movies_df.withColumn("year", col("title").substr(-5, 4))


In [13]:
movies_df.show()

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|      1|    Toy Story (1995)|Adventure|Animati...|1995|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|1995|
|      5|Father of the Bri...|              Comedy|1995|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|
|      8| Tom and Huck (1995)|  Adventure|Children|1995|
|      9| Sudden Death (1995)|              Action|1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|1995|
|     11|American Presiden...|Comedy|Drama|Romance|1995|
|     12|Dracula: Dead and...|       Comedy|Horror|1995|
|     13|        Balto (1995)|Adventure|Animati...|1995|
|     14|        Nixon (1995)|               Drama|1995|
|     15|Cutthroat Island ...|A

In [14]:
genres_year_count = movies_df.withColumn("genre", explode(split(col("genres"), "\\|"))) \
    .groupBy("year", "genre").count()

In [15]:
genres_year_count.show()

+----+------------------+-----+
|year|             genre|count|
+----+------------------+-----+
|1994|             Crime|   66|
|1967|             Drama|  171|
|1958|             Drama|  156|
|1941|           Mystery|   26|
|1984|            Comedy|  162|
|1997|          Children|   78|
|1998|               War|   26|
|1978|         Adventure|   45|
|1973|           Musical|   10|
|1922|           Mystery|    3|
|2003|           Musical|   19|
|1966|            Horror|   41|
|2006|         Adventure|  116|
|2008|           Fantasy|   95|
|1925|             Crime|    3|
|1912|            Comedy|   10|
|010)|               War|    1|
|1928|(no genres listed)|    5|
|1950|(no genres listed)|   24|
|arit|(no genres listed)|    1|
+----+------------------+-----+
only showing top 20 rows



In [207]:
indexer = StringIndexer(inputCol="year", outputCol="year_index",handleInvalid="keep")
assembler = VectorAssembler(inputCols=["year_index", "title_length"], outputCol="features")

# Calculate title length
from pyspark.sql.functions import length


In [208]:
print(indexer)

StringIndexer_e1548e6c89b3


In [209]:
movies_df = movies_df.withColumn("title_length", (length(col("title")) - 7))  # Assuming the year is always 4 characters (e.g., "(1995)")


In [211]:
indexer2 = StringIndexer(inputCol="genre", outputCol="genre_index")

In [212]:
# movies_df = assembler.transform(movies_df)
movies_df.show()

+-------+--------------------+--------------------+----+------------+-----------+
|movieId|               title|              genres|year|title_length|genre_index|
+-------+--------------------+--------------------+----+------------+-----------+
|      1|    Toy Story (1995)|Adventure|Animati...|1995|           9|      140.0|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|           7|      102.0|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|          16|        7.0|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|1995|          17|        9.0|
|      5|Father of the Bri...|              Comedy|1995|          27|        2.0|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|           4|       31.0|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|           7|        7.0|
|      8| Tom and Huck (1995)|  Adventure|Children|1995|          12|       71.0|
|      9| Sudden Death (1995)|              Action|1995|          12|       15.0|
|     10|    Gol

In [213]:
# from pyspark.ml.feature import PCA
# pca = PCA(k=2, inputCol="features", outputCol="pca_features")
dt = DecisionTreeClassifier(labelCol="genre_index", featuresCol="features", maxDepth=5)


In [226]:
# dt = DecisionTreeClassifier(featuresCol="features", labelCol="genre_index")
rf = RandomForestClassifier(featuresCol="features", labelCol="genre_index",maxBins=800)

In [227]:
pipeline_dt = Pipeline(stages=[dt])

pipeline_rf = Pipeline(stages=[indexer, assembler, rf])

In [228]:
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10]) \
    .addGrid(rf.numTrees, [10, 20]) \
    .build()

In [229]:
evaluator = MulticlassClassificationEvaluator(labelCol="genre_index", predictionCol="prediction", metricName="accuracy")


In [230]:
crossval_dt = CrossValidator(estimator=pipeline_dt,
                             estimatorParamMaps=paramGrid,
                             evaluator=evaluator,
                             numFolds=5)
crossval_rf = CrossValidator(estimator=pipeline_rf,
                             estimatorParamMaps=paramGrid,
                             evaluator=evaluator,
                             numFolds=5)

In [231]:
# dt_model = crossval_dt.fit(movies_df)
rf_model = crossval_rf.fit(movies_df)

24/04/17 02:58:48 WARN DAGScheduler: Broadcasting large task binary with size 1028.9 KiB
24/04/17 02:58:50 WARN DAGScheduler: Broadcasting large task binary with size 1138.2 KiB
24/04/17 02:58:53 WARN DAGScheduler: Broadcasting large task binary with size 1515.1 KiB
24/04/17 02:58:55 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
24/04/17 02:58:57 WARN DAGScheduler: Broadcasting large task binary with size 1264.6 KiB
24/04/17 02:58:59 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
24/04/17 02:59:02 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
24/04/17 02:59:04 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
24/04/17 02:59:07 WARN DAGScheduler: Broadcasting large task binary with size 1514.7 KiB
24/04/17 02:59:09 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
24/04/17 02:59:11 WARN DAGScheduler: Broadcasting large task binary with size 1731.1 KiB
24/04/17 02:59:13 WARN DAGScheduler:

In [232]:
predictions_dt = rf_model.transform(movies_df)


In [233]:
indexer_model = StringIndexer(inputCol="genres", outputCol="genre_index2").fit(movies_df)


In [234]:
predictions_dt = indexer_model.transform(predictions_dt)


In [235]:
confusion_matrix_dt = confusion_matrix(predictions_dt.select("genre_index2").collect(), predictions_dt.select("prediction").collect())


24/04/17 03:18:30 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB


In [236]:
accuracy_dt = accuracy_score(predictions_dt.select("genre_index2").collect(), predictions_dt.select("prediction").collect())


24/04/17 03:19:20 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB


In [237]:
precision_dt = precision_score(predictions_dt.select("genre_index2").collect(), predictions_dt.select("prediction").collect(), average='macro')


24/04/17 03:20:01 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [238]:
recall_dt = recall_score(predictions_dt.select("genre_index2").collect(), predictions_dt.select("prediction").collect(), average='macro')


24/04/17 03:20:40 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB


In [239]:
f1_dt = f1_score(predictions_dt.select("genre_index2").collect(), predictions_dt.select("prediction").collect(), average='macro')


24/04/17 03:20:59 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB


In [240]:
print("Confusion Matrix - Decision Tree:")
print(confusion_matrix_dt)
print("Accuracy - Decision Tree:", accuracy_dt)
print("Precision - Decision Tree:", precision_dt)
print("Recall - Decision Tree:", recall_dt)
print("F1 Score - Decision Tree:", f1_dt)

Confusion Matrix - Decision Tree:
[[10997  1113    53 ...     0     0     0]
 [ 5378  2528    19 ...     0     0     0]
 [ 6218  1282   123 ...     0     0     0]
 ...
 [    1     0     0 ...     0     0     0]
 [    1     0     0 ...     0     0     0]
 [    1     0     0 ...     0     0     0]]
Accuracy - Decision Tree: 0.16350231692801923
Precision - Decision Tree: 0.001168727881152141
Recall - Decision Tree: 0.0007375331621249865
F1 Score - Decision Tree: 0.0004023924658307174
